# Lesson 1: Machine learning fundamentals

*Teachers:* Fares Schulz, Lina Campanella\
*Date:* 16.10.2025


In this course we will cover 
1. A recap on the concept of machine learning
2. Creating our own dataset
3. A detailed implementation of simple linear regression
4. A detailed implementation of simple binary classification
5. An explanation on Train–Test Split and Model Evaluation
6. Two exercises to explore polynomial regression and Normal Equations for linear regression

This script was inspired by the Creative Machine Learning course of Philippe Esling



# Defining Machine Learning

Machine learning aims to approximate an unknown mapping between input and output spaces, $\mathcal{X} \mapsto \mathcal{Y}$,
given only a finite set of Data
$
\mathcal{D} = \{(x_i, y_i)\}_{i=1}^{N},
$
where $(x_i \in \mathcal{X})$ and $(y_i \in \mathcal{Y})$.

We define a **parametric model**
$ f_{\boldsymbol{\theta}} \in \mathcal{F}$,
with parameters $(\boldsymbol{\theta} \in \boldsymbol{\Theta})$, chosen such that
$
f_{\boldsymbol{\theta}}(x) \approx y$.


Any machine learning problem involves these core components:
1. **Dataset** $(\mathcal{D})$: A collection of input-output pairs $ \mathcal{D} = \{(x_i, y_i)\}_{i=1} $, The dataset must be representative of the true underlying function $(f^*: \mathcal{X} \mapsto \mathcal{Y})$  
2. **Function or Model** $(f_{\boldsymbol{\theta}})$: A parameterized function that maps inputs $\mathbf{x}_i$ to predicted outputs $\mathbf{\hat{y}}_i = f_{{\theta}}(\mathbf{x_i})$ The choice of function defines the function space $\mathcal{F}_{\Theta}$
3. **Parameters ($\theta$)**: The set of parameters that define the specific function within the function space. These parameters are adjusted during training to minimize the empirical risk. come from the parameter space $\boldsymbol{\Theta}$
4. **Lossfunction**: A function that quantifies the difference between the predicted outputs $\mathbf{\hat{y}}_i$ and the true labels $\mathbf{y}_i$. The choice of loss function depends on the task (e.g., regression vs. classification).
5. **Optimization Algorithm**: A method for adjusting the parameters $\boldsymbol{\theta}$ to minimize the empirical risk $\hat{R}(\theta)$



## Linear Regression

To understand how a machine learning model works mathematically, we start with the case of simple linear regression, where $\mathbf{x}\in\mathbb{R}$. This implies that our model will follow
$$f_{\theta}(x) = \theta_0 + \theta_1 x$$

We will measure the errors made by our model by using the Mean Squared Error (MSE) loss, defined as 
$$\mathcal{L}_{MSE}(\theta) = \frac{1}{N}\sum_{i=1}^{N} ( y_{i} - f_{\theta}(x_i))^{2} = \frac{1}{N} \sum_{i=1}^{N}  ( y_{i} - (\theta_{0} + \theta_{1} x_{i}) )^{2}$$

Then our goal is to find the most adequate set of parameters $\theta = \{\theta_{0}, \theta_{1}\}$, which are those that minimize the MSE loss defined previously. To do so, we will implement the gradient descent algorithm.

### Manual implementation in NumPy

We start by performing a full manual implementation, in the sense that we need to manually derive the gradient in order to apply the gradient descent updates. To do so, we will rely on Numpy.

We start by importing libraries and set a random seed to ensure that the random number generator produces always a reproducible series of random numbers.

**Used functions**
- `np.random.seed`: sets the seed for the NumPy random number generator. [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html)

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

np.random.seed(42)

### Generate a synthetic dataset.

For the sake of this exercise, we will generate the data ourselves, so that we know the true values that we are looking for in advance. As in most real-world cases, our measurements are not perfect, they contain some amount of **observation noise** (also called *residual error*) $\epsilon$ . Hence, we define a linear relationship following
$$y_i = f^*(\mathbf{x}_i) + \epsilon_i$$

**Useful functions**
- `np.random.rand`: generates an array of random numbers uniformly distributed over [0, 1). [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html)
- `np.random.randn`: generates an array of random numbers from the standard normal distribution (mean 0, variance 1). [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)
- `np.random.uniform`" draws samples from a uniform distribution. [Documentation](https://numpy.org/doc/2.1/reference/random/generated/numpy.random.uniform.html)

Below, we will visualize synthetic data with generated noise levels.

In [175]:
# Creating a synthetic dataset

true_theta0 = 4
true_theta1 = -3
n_obs = 100

epsilon = np.random.randn(n_obs) * 0.3


In [ ]:
def generate_data(eps=0.7):
    x = np.linspace(0, 1, n_obs) 
    y =  true_theta1 * x + true_theta0 +  eps * epsilon
    return x, y

def plot_data(eps=0.7):
    x, y = generate_data(eps)
    plt.figure(figsize=(7,4))
    plt.scatter(x, y, color="C0", label="Created data")
    plt.plot(x, (true_theta1*x + true_theta0), color="C1", label="True function" )
    plt.title(rf"$y = {true_theta1:.2f}*x + {true_theta0:.2f} + ϵ, ϵ \in[-{eps:.2f},{eps:.2f}]$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim(0, 1)  
    plt.ylim(0, 5)
    plt.grid(True)
    plt.axhline(0, color='black', lw=1.5) 
    plt.axvline(0, color='black', lw=1.5)
    plt.legend()
    plt.show()

# Plot with interactive slider for epsilon
interact(
    plot_data,
    eps=widgets.FloatSlider(value=0.3, min=0.0, max=2, step=0.05, description="ϵ range"),
)


In [ ]:
# Data set for the linear regression with set epsilon
true_theta0 = 2
true_theta1 = 3
n_obs = 100
epsilon = np.random.randn(n_obs) * 0.2

x = np.linspace(0, 1, n_obs)
y = (true_theta1 * x + true_theta0) + epsilon

### Computing the MSE loss

The next step is to calculate the errors made by our model by using the Mean Squared Error (MSE) loss, defined as 
$$\mathcal{L}_{MSE}\left( \hat{\mathbf{y}},\theta \right) = \sum_{i=1}^{n} ( \hat{y}_{i} - y_{i} )^{2} = \sum_{i=1}^{n} ( (\theta_{0} + \theta_{1} x_{i}) -  y_{i} )^{2}$$

This will allow us to evaluate the performances of our model, but is also the basis for the gradient descent algorithm.


### Implement the gradient descent algorithm.

First we have to Iinitialize the parameters $\theta_1$ and $\theta_0$ to small random values
1. Evaluate the predictions made by our model 
$$\hat{y} = \theta_{1}x + \theta_0$$
2. Compute the mean squared error (MSE) loss between the predicted values and the ground truth labels: 
$$\mathcal{L}_{MSE} = \frac{1}{n} \sum_{i=1}^n (\hat{y_i} - y_i)^2$$
3. Compute the gradients of the loss with respect to the parameters: 
$$\frac{\partial \mathcal{L}}{\partial \theta_1} = \sum_{i} 2 * (\hat{y}_{i} - y_{i})*x_{i}$$
$$\frac{\partial \mathcal{L}}{\partial \theta_0} = \sum_{i} 2 * (\hat{y}_{i} - y_{i})$$
4. Update the parameters using the gradients and a learning rate $\eta$: 
$$\theta_1 \leftarrow \theta_1 - \eta \frac{\partial \mathcal{L}}{\partial \theta_1}$$

$$\theta_0 \leftarrow \theta_0 - \eta \frac{\partial \mathcal{L}}{\partial \theta_0}$$

In [ ]:
# Mean Squared Error loss function
def mse_loss_np(y, y_pred):
    return   np.sum((y_pred - y) ** 2)/n_obs

# Manually calculated gradients of the MSE loss function
def mse_loss_gradient_np(y,y_pred, x):
    dLdtheta1 = 2 * np.sum((y_pred - y) * x)
    dLdtheta0 = 2 * np.sum((y_pred - y))
    return dLdtheta0, dLdtheta1

# Optimization with gradient descent
def gradient_descent_np(x, y, lr, n_iter):
    #  Initializing the parameters
    theta1 = np.random.randn()
    theta0 = np.random.randn()

    loss_history = []

    # Gradient Descent Loop 
    for i in range(n_iter):
        # Predictions
        y_pred = theta1 * x + theta0
        
        # Compute gradients
        dLdtheta0, dLdtheta1 = mse_loss_gradient_np(y,y_pred,x)

        # Update parameters
        theta1 = theta1 - lr * dLdtheta1
        theta0 = theta0 - lr * dLdtheta0

        # Compute Mean Squared Error loss
        loss_history.append(mse_loss_np(y, y_pred))

    return theta0, theta1, loss_history

In [ ]:
# Defining the number of iterations and learning rate
n_iters = 200
lr = 1e-3

theta0, theta1, loss_history = gradient_descent_np(x, y, lr, n_iters)

# Plot data and fitted line
plt.figure(figsize=(8,5))
plt.scatter(x, y, color="C0", label="Data")
plt.plot(x, true_theta1 * x + true_theta0, color="C1", lw=2, label="True linear regreassion")
plt.plot(x, theta1 * x + theta0, color="C2", lw=2, linestyle="--", label=rf"Learned model ($\theta_1$={theta1:.2f}, $\theta_0$={theta0:.2f})")
plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Linear Regression with Gradient Descent ({n_iters} iterations)")
plt.grid(True)
plt.legend()
plt.show()

# Plot loss over iterations
plt.figure(figsize=(6,4))
plt.plot(loss_history, color="C3", lw=2)
plt.xlabel("Iteration")
plt.ylabel("MSE Loss")
plt.title("Loss vs Iteration")
plt.grid(True)
plt.show()


### Implementation with Jax

The previous implementation required us to perform manual differentiation of our loss function to understand how to update the parameters. However, large developments have been made in the field of automatic differentiation. 

The recent library [JAX](https://github.com/google/jax) extends NumPy with this automatic differentiation feature, while providing a functional approach to numerical computing, allowing for easy gradient computation. Its ability to handle complex and custom gradients makes JAX particularly well-suited for advanced research projects. Similar to NumPy, we strongly encourage you to learn JAX through the set of [tutorials](https://jax.readthedocs.io/en/latest/).

Note that `JAX` has been thought as an extension of `NumPy`, therefore an extremely large portion of its API simply mirrors the `NumPy` functions by adding automatic differentiation features to it.

**Useful functions**
- `random.PRNGKey`: function to generate a key for the pseudorandom number generator. [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.PRNGKey.html)
- `jit`: function to compile a function for faster execution [Documentation](https://jax.readthedocs.io/en/latest/jit.html)
- `grad`: function to compute the gradient of a function [Documentation](https://docs.jax.dev/en/latest/_autosummary/jax.grad.html#jax.grad)

In [177]:
import jax.numpy as jnp
from jax import grad, random

key = random.PRNGKey(42)

### Dataset

We can simply keep our previous approach to generating the dataset but we will rely on `jnp.ndarray` instead of `np.ndarray`. 

**Useful functions**
- `random.split`: function to split a PRNGKey into a list of subkeys [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.split.html)
- `random.uniform`: function to generate an array of uniformly distributed random numbers [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.uniform.html)
- `random.normal`: function to generate an array of normally distributed random numbers [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.normal.html)

In [ ]:
# Splits the key and uses the keys to create data points
key_0, key_1, key = random.split(key, 3)
x = random.uniform(key_0, (100,))
y = true_theta0 + true_theta1 * x + (random.normal(key_1, (100,)) * 0.2)

### Loss function

Similarly we have to define the loss function and its gradient.

**Useful function**
- `jnp.mean`: function to compute the mean of an array [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.mean.html)

In [ ]:
# Mean Squared Error loss function
def loss_function_jax(x, y, theta0, theta1):
    y_pred = theta1 * x + theta0
    return jnp.mean((y_pred - y)**2)

# Optimization with gradient descent using jax's grad function 
def gradient_descent_jax(key, x, y, lr, n_iter):
    # Initializing the parameters
    k_0, k_1 = random.split(key, 2)
    theta0 = random.normal(k_0, (1,))
    theta1 = random.normal(k_1, (1,))
    loss_history = []

    grad_loss_function = grad(loss_function_jax, argnums=[2, 3])


    for i in range(n_iter):
        # Compute gradients
        dLdtheta = grad_loss_function(x, y, theta0, theta1)
        
        # Update the parameters
        theta0 -= lr * dLdtheta[0]
        theta1 -= lr * dLdtheta[1]

        # Compute Mean Squared Error loss
        loss_history.append(loss_function_jax(x, y, theta0, theta1))

    return theta0, theta1, loss_history



In [ ]:
# Defining the number of iterations and learning rate
n_iter = 200
lr = 0.05

theta0, theta1, loss_history = gradient_descent_jax(key,x, y, lr, n_iters)

# Plot data and fitted line
plt.figure(figsize=(8,5))
plt.scatter(x, y, color="C0", label="Data")
plt.plot(x, (true_theta1 * x + true_theta0), color="C1", lw=2, label="True linear regreassion")
plt.plot(x, theta1 * x + theta0, color="C2", lw=2, linestyle="--", label=rf"Learned model ($\theta_1$={theta1[0]:.2f}, $\theta_0$={theta0[0]:.2f})")
plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Linear Regression with Gradient Descent ({n_iters} iterations)")
plt.grid(True)
plt.legend()
plt.show()

# Plot loss over iterations
plt.figure(figsize=(6,4))
plt.plot(loss_history, color="C3", lw=2)
plt.xlabel("Iteration")
plt.ylabel("MSE Loss")
plt.title("Loss vs Iteration")
plt.grid(True)
plt.show()


## Binary Classification

In binary classification, the goal is to assign each input $ \mathbf{x} $ in $ \mathbb{R}^d $ to one of two possible classes, typically represented as $ y \in \{-1, +1\} $.

For this exercise we will use a linear classifier model. The relationship between inputs and outputs use a linear decision function:

$$
f(\mathbf{x}) = \theta_0 + \theta_1 x_1 + \theta_2 x_2
$$

The predicted label is obtained from the sign of this function:

$$
\hat{y} = \text{sign}(f_{\theta}(\mathbf{x}))
$$

The parameters $ \boldsymbol{\theta} = (\theta_0, \theta_1, \theta_2) $ are estimated by minimizing the Hinge loss function that penalizes incorrect or uncertain predictions.  





In [ ]:
# Creating a new data set
n_obs = 100
epsilon = np.random.randn(n_obs) * 2

# Input
x0 = np.linspace(0, 1.1, n_obs)
x1 = np.linspace(0.5,2, n_obs)

# Output 
y0 = -3 * x0**3 + 13 + epsilon
y1 = 2 * x1**3 + 2 + epsilon

X = np.vstack([
    np.stack([x0, y0], axis=1),
    np.stack([x1, y1], axis=1)
])
Y = np.concatenate([
    -np.ones_like(x0),
    np.ones_like(x1)
])


### Hinge Loss

The Hinge Loss function is Loss function that is commonly used in binary classification models. 



For a single data point $(x_i, y_i)$, where $ y_i \in \{-1, +1\} $, the hinge loss is defined as:

$$
\mathcal{L_{hinge}} = \frac{1}{N} \sum \max(0, 1 - y_i f_{\theta}(x_i))
$$

- If the prediction $\hat{y}$ has the same sign as $ y_i $ and its magnitude is ≥ 1, then the loss is 0: the point is correctly classified and outside the margin.  
- If the prediction is incorrect or within the margin, the loss increases linearly as $f(x_i)$ moves away from the correct side.


This time we will use Stochastic Gradient Descent (SGD) as the optimazation algorithm. With SGD parameters are updated using gradient from a single randomly selected sample per iteration.

### Gradient Descent vs. Stochastic Gradient Descent (SGD)

Both gradient descent and stochastic gradient descent (SGD) are optimization algorithms used to minimize a loss function by iteratively updating model parameters in the direction of the negative gradient.

**Gradient Descent**
- Uses all training samples to compute the gradient at each iteration.  
- Provides a precise estimate of the true gradient.  
- Updates are smooth and stable, but computation can be expensive for large datasets.  
- The parameter update rule is:

  $$
  \boldsymbol{\theta_{t+1}} = \boldsymbol{\theta_t} - \eta \nabla_\theta \mathcal{\hat{R}}(\boldsymbol{\theta_t}) = \boldsymbol{\theta_t} - \frac{\eta}{N} \displaystyle\sum_{i=1}^N \nabla_\theta \ell(f_{\theta_t}(x_{i_t}) , y_{i_t})
  $$

  where $ \eta $ is the learning rate and $ \ell$ is a single sample loss function.

**Stochastic Gradient Descent (SGD)**
- Uses one randomly chosen training sample per iteration to approximate the gradient.  
- Each update is fast, but the gradient estimate is noisy.  
- The loss fluctuates more during training but still tends to decrease on average.  
- This noise can help escape shallow local minima and improve generalization.

The update rule becomes:

  $$
  \theta_{t+1} = \theta_t - \eta \nabla_\theta \ell(f_{\theta_t}(x_{i_t}) , y_{i_t})
  $$



<!-- #### Summary

| Method | Gradient Computation | Stability  Computation per Step | Typical Use |
|---------|----------------------|------------|----------------------|--------------|
| Gradient Descent | Full dataset | Smooth, stable | High | Small datasets |
| Stochastic Gradient Descent | One sample | Noisy, oscillatory | Very low | Large datasets |

In practice, many modern algorithms use a compromise called **mini-batch SGD**, which computes the gradient on a small random subset of the data to balance efficiency and stability. -->


### Numpy implementation

In [ ]:
# Hinge loss function
def hinge_loss_np(y, y_pred):
    losses = 1 - y * y_pred
    losses = np.maximum(0, losses)
    return np.mean(losses)

# Manually calculated gradients of the Hinge loss function 
def hinge_gradient_np(x_i, y_i, theta0, theta1, theta2):
    f_i = theta0 + theta1 * x_i[0] + theta2 * x_i[1]
    if 1 - y_i * f_i > 0:  # only if violating margin
        dtheta0 = -y_i
        dtheta1 = -y_i * x_i[0]
        dtheta2 = -y_i * x_i[1]
    else:
        dtheta0 = dtheta1 = dtheta2 = 0.0
    return dtheta0, dtheta1, dtheta2

# Optimization with stochastic gradient descent using numpy 
def stochastic_gradient_descent_np(x, y, lr=0.05, n_iter=1000):
    theta0 = np.random.randn()
    theta1 = np.random.randn()
    theta2 = np.random.randn()

    loss_history = []

    for i in range(n_iters):
        i_t = np.random.randint(len(x))
        x_i = x[i_t]
        y_i = y[i_t]

        
        # Compute loss (for monitoring)
        y_pred_full = theta0 + theta1 * x[:,0] + theta2 * x[:,1]
        loss_history.append(hinge_loss_np(y, y_pred_full))


        # Gradients
        dLdtheta0, dLdtheta1, dLdtheta2 = hinge_gradient_np(x_i, y_i, theta0, theta1, theta2)

        # Update parameters
        theta0 -= lr * dLdtheta0
        theta1 -= lr * dLdtheta1
        theta2 -= lr * dLdtheta2
        theta = [theta0,theta1,theta2]

    return theta, loss_history



In [ ]:
# Defining the number of iterations and learning rate
n_iters = 5000
lr = 0.003

theta, loss_history = stochastic_gradient_descent_np(X, Y, lr, n_iters)

theta0, theta1, theta2 = theta

x_line = np.linspace(X[:,0].min(), X[:,0].max(), 100)
y_line = -(theta0 + theta1 * x_line ) / theta2

# Plot data and fitted line
plt.figure(figsize=(8,5))
plt.scatter(x0, y0, color="C0", label="Class 0")
plt.scatter(x1, y1, color="C1", label="Class 1")
plt.plot(x_line, y_line, color="black", lw=2, label=rf"sign boundary ($\theta_0 = {round(theta0,2)}$, $\theta_1 = ${round(theta1,2)}, $\theta_2 = ${round(theta2,2)})")
plt.xlabel("x")
plt.ylabel("y")
plt.title(rf"Binary classification with Gradient Descent ({n_iters} iterations)")
plt.grid(True)
plt.legend()
plt.show()

# Plot loss over iterations
plt.figure(figsize=(6,4))
plt.plot(loss_history, color="C3", lw=2)
plt.xlabel("Iteration")
plt.ylabel("MSE Loss")
plt.title("Loss vs Iteration")
plt.grid(True)
plt.show()


### Jax Implementation

We will repeat the same exercize using the JAX library to get a better understanding of JAX.

In [106]:
key = random.PRNGKey(42)
epsilon = random.normal(key, (n_obs,)) * 2

In [ ]:
# Hinge loss function  
def hinge_loss_jax(theta, x, y):
    theta0, theta1, theta2 = theta
    f = theta0 + theta1 * x[:, 0] + theta2 * x[:, 1]
    losses = 1 - y * f
    losses = jnp.maximum(0, losses)
    return jnp.mean(losses)

# Compute gradient for one-sample loss
def single_loss_jax(theta, x_i, y_i):
    f_i = theta[0] + theta[1] * x_i[0] + theta[2] * x_i[1]
    return jnp.maximum(0, 1 - y_i * f_i)

# Optimization with stochastic gradient descent using jax's grad
def stochastic_gradient_descent_jax(key, x, y, lr=0.05, n_iter=1000):
    theta = random.normal(key, (3,))  # [theta0, theta1, theta2]
    
    loss_history = []
    for i in range(n_iters):
        # Sample one random observation
        key, subkey = random.split(key)
        i_t = random.randint(subkey, shape=(), minval=0, maxval=len(x))
        x_i = x[i_t]
        y_i = y[i_t]


        grads = grad(single_loss_jax)(theta, x_i, y_i)

        # Parameter update
        theta = theta - lr * grads

        # Compute full loss (for monitoring)
        loss_history.append(hinge_loss_jax(theta, x, y))

    return theta, loss_history
    

In [ ]:
# Defining the number of iterations and learning rate
n_iters = 5000
lr = 0.003

theta, loss_history = stochastic_gradient_descent_jax(key, X, Y, lr, n_iters)

theta0, theta1, theta2 = theta

x_line = jnp.linspace(X[:,0].min(), X[:,0].max(), 100)
y_line = -(theta0 + theta1 * x_line) / theta2

# Plot data and fitted line
plt.figure(figsize=(8,5))
plt.scatter(x0, y0, color="C0", label="Class 0")
plt.scatter(x1, y1, color="C1", label="Class 1")
plt.plot(x_line, y_line, color="black", lw=2, label="Decision boundary")
plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Binary Classification with SGD (JAX, {n_iters} iterations)")
plt.grid(True)
plt.legend()
plt.show()

# Plot loss over iterations
plt.figure(figsize=(6,4))
plt.plot(loss_history, color="C3", lw=2)
plt.xlabel("Iteration")
plt.ylabel("Hinge Loss")
plt.title("Training Loss (SGD, JAX)")
plt.grid(True)
plt.show()

## Train–Test Split and Model Evaluation

To properly assess how well a machine learning model generalizes, we must evaluate it on data it has not seen during training.  
If we only measure performance on the training data, there is a risk of overfitting the model. Overfitting the model means using overly complex models (e.g., high-degree polynomials) can lead to fitting noise rather than the underlying pattern, resulting in poor generalization to new data.

<!-- A standard approach is to split the available dataset into three parts:
- **Training set:** used to fit the model parameters.
- **Test set:** used only for evaluating the model after training.
- **Validation set:** used to test how well the  -->

The model’s performance on the test set provides an estimate of how well it will perform on new, unseen data.

Below, we’ll split our dataset from the first exercize about linear regression, train the model on the training data only, and compare the loss on both the training and validation sets.


In [179]:
# Splitting our dataset 80/20
train_size = int(0.8 * len(x))

x_train, x_valid = x[:train_size], x[train_size:]
y_train, y_valid = y[:train_size], y[train_size:]


In [181]:
n_iters = 200
lr = 0.03

theta0, theta1, loss_history = gradient_descent_jax(key, x_train, y_train, lr, n_iters)

print(rf"Gradient Descent coefficients: theta0 = {round(theta0,2)}, theta1 = {theta1}" )



Gradient Descent coefficients: theta0 = [2.1299999], theta1 = [2.7517393]


In [186]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

y_pred_train = theta0 + theta1 * x_train
y_pred_valid = theta0 + theta1 * x_valid 
mse_train = mean_squared_error(y_train, y_pred_train)
mse_valid = mean_squared_error(y_valid, y_pred_valid)

print("MSE for Gradient Descent for the training data:", mse_train)
print("MSE for Gradient Descent for the validation data:",  mse_valid)

MSE for Gradient Descent for the training data: 0.040729832
MSE for Gradient Descent for the validation data: 0.036479842


## Exercises

### Exercise 1 - Polinomial Regression

In this exercise, you will implement polynomial regression using NumPy.

In [188]:
from typing import Callable, List, Tuple

def polynomial_regression_dataset(
        n_observations: int     = 200,
        noise: float            = 0.2,
        c1_center: List[float]  = [-2, -1],
        c2_center: List[float]  = [2, 1]
    ) -> Tuple[np.ndarray, np.ndarray]:
    ######################
    # YOUR CODE GOES HERE
    ######################
    return ...

In [ ]:
def mse_loss_np(
    y: np.ndarray, 
    y_bar: np.ndarray
  ) -> float:
  """
    Numpy-based function to compute the _normalized_ MSE loss.
    Input:
        y       : (np.ndarray) array of groundtruth classes
        y_pred   : (np.ndarray) array of approximate classes
    Returns:
        loss    : (float) value of the MSE
  """
  ######################
  # YOUR CODE GOES HERE
  ######################
  return loss

def infer_np(
        x: np.ndarray,
        theta0: float, 
        tetha1: float,
        degree: int
    ) -> np.ndarray:
    """
    Numpy-based function to classify input points (model function)
    Input:
        x       : (np.ndarray) array of input observations
        theta0      : (float) bias
        theta1      : (float) coefficient weight 1
        theta2      : (float) coefficient weight 2
    Returns:
        y_bar   : (np.ndarray) array of infered classes (-1 / +1)
    """
    ######################
    # YOUR CODE GOES HERE
    ######################
    return y_pred

def gd_regression_np(
    x: np.ndarray,
    y: np.ndarray, 
    n_iter: int, 
    lr: float, 
    degree: int,
    loss_func: Callable[[np.ndarray, np.ndarray], np.ndarray], 
    seed: int = 0
  ) -> Tuple[float, float, float, List[float]]:
  """
    This function performs gradient descent for linear classification model.
    Input:
        x               : (np.ndarray) an array of observations
        y               : (np.ndarray) an array of classes corresponding to the observations
        n_iter          : (int) number of iterations
        lr              : (float) learning rate
        degree          : (int) polynomial degree
        loss_function   : (function) loss function
        seed            : (int) random number generator seed
    Ouput:
        theta0              : (float) bias
        tetha1              : (float) coefficient weight 1
        loss_history    : (list) an array of loss values obtained from each iteration
  """
  ######################
  # YOUR CODE GOES HERE
  ######################
  return theta0, theta1, loss_history

### Exercise 2 - Normal Equations for linear regression
Instead of using gradient descent to find $\boldsymbol{\theta}$, we can derive a closed-form solution by setting the gradient of the MSE loss to zero. This leads to the Normal Equation:
$$
\boldsymbol{\theta}^* = (\mathbf{X}^T\mathbf{X})^{-1} \mathbf{X}^T\mathbf{y}
$$

1. Create a dataset:
    Create a simple linear relationship between $x$ and $y$: $ y = 4x + 1 + \epsilon $.
    Use 50 samples.

2. Construct the design matrix

3. Implement the Normal Equation

4. Make predictions

5. Visualize the results**
   - Plot the original data points and the fitted regression line.
   - Optionally compute and print the **Mean Squared Error (MSE)** of your model.


In [187]:
######################
# YOUR CODE GOES HERE
######################